In [1]:
ORIGIN = "BARCELONA (TODAS)"
DESTINATION = "MADRID (TODAS)"
FECHA_IDA_SEL = "24/03/2023"

In [2]:
HEADLESS_MODE = False
RENFE_URL = 'https://www.renfe.com/es/es'

In [3]:
# https://github.com/gerardcl/renfe-cli
# https://github.com/grcanosa/renfe-notifier-bot/tree/e6b0438557732b541ea629bec2aaa4479308436c
# https://www.renfe.com/es/es/viajar/informacion-util/horarios

## Library Requirements

In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import datetime
import time
import pprint
from bs4 import BeautifulSoup

## Create WebDriver in Chrome

In [5]:
if HEADLESS_MODE == True:
    op = webdriver.ChromeOptions()
    op.add_argument('headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=op)
else:
    driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver, 30)



====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST chromedriver version for 111.0.5563 google-chrome
Driver [/Users/danielfiuzadosil/.wdm/drivers/chromedriver/mac64/111.0.5563.64/chromedriver] found in cache
/var/folders/tl/yx271fpd4sd_d224zzcb65840000gp/T/ipykernel_28346/655600415.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Open Renfe URL

In [6]:
driver.get(RENFE_URL)
driver.implicitly_wait(10)

## Reject Cookies

In [7]:
wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='onetrust-reject-all-handler']"))).click()

## Complete Origin

In [8]:
# Cliq on Origin box
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="origin"]'))).click()

In [9]:
# Rellenar Origin Value
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="origin"]'))).send_keys(ORIGIN)

In [10]:
# Cliq in the fist shown option
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="awesomplete_list_1_item_0"]'))).click()

## Complete Destination

In [11]:
# Cliq on Destino box
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="destination"]'))).click()

In [12]:
# Rellenar Hasta Value
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="destination"]'))).send_keys(DESTINATION)

In [13]:
# Cliq in the fist shown option
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="awesomplete_list_2_item_0"]'))).click()

## Select One-Way Only

In [14]:
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tripType"]/div/button'))).click()
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tripType"]/div/div/ul/li[1]/button'))).click()

## Select Travel Date

In [15]:
# Cliq on Calendar box
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="datepicker"]/div/input'))).click()

In [16]:
FECHA_IDA_SEL

'24/03/2023'

In [17]:
# Calculate date and find element on calendar
# 
date = datetime.datetime.strptime(FECHA_IDA_SEL, '%d/%m/%Y')
date_unixtime = int(time.mktime(date.timetuple()))
date_unixtime_str = str(date_unixtime) + '000'
# 
xpath_search = "//*[contains(@data-time, '" + str(date_unixtime_str) + "')]"
element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, xpath_search)))

In [18]:
# Click on the right date on the calendar
element.click()

In [19]:
# Click on the "Accept"
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="datepicker"]/section/div[2]/button[2]'))).click()

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (898, 733)
  (Session info: chrome=111.0.5563.64)
Stacktrace:
0   chromedriver                        0x00000001048fb428 chromedriver + 4899880
1   chromedriver                        0x0000000104878a23 chromedriver + 4364835
2   chromedriver                        0x00000001044c2bf6 chromedriver + 474102
3   chromedriver                        0x000000010450fd81 chromedriver + 789889
4   chromedriver                        0x000000010450cdde chromedriver + 777694
5   chromedriver                        0x0000000104509a54 chromedriver + 764500
6   chromedriver                        0x000000010450843f chromedriver + 758847
7   chromedriver                        0x00000001044f8cf3 chromedriver + 695539
8   chromedriver                        0x000000010452c562 chromedriver + 906594
9   chromedriver                        0x00000001044f8361 chromedriver + 693089
10  chromedriver      

## Search for Tickets

In [20]:
# Click on the "Search for Tickets"
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="contentPage"]/div/div/div[1]/div/div/div/div/div/div/rf-header/rf-header-top/div[2]/rf-search/div/div[2]/div[2]/div[2]/form/rf-button/div/div/button/div[2]'))).click()

## Extract Data

### RAW Data HTML

In [21]:
time.sleep(10)
beautifulSoupText = BeautifulSoup(driver.page_source, 'lxml')

In [22]:
print(beautifulSoupText.body.prettify())

<body onload="">
 <div class="container" id="wrapper">
  <h1 class="sr-only">
   Lista de Trenes
  </h1>
  <input id="tiempoTrenesVuelta" name="tiempoTrenesVuelta" type="hidden" value="1000"/>
  <script src="/vol/CEX/js/header.js?v=5" type="text/javascript">
  </script>
  <script src="/vol/CEX/js/steinciljs/renfe-web-components.esm.js?v=5" type="module">
  </script>
  <script nomodule="" src="/vol/CEX/js/steinciljs/renfe-web-components.js?v=5">
  </script>
  <link href="/vol/CEX/js/steinciljs/renfe-web-components-1639498827542.css?v=5" rel="stylesheet"/>
  <script src="https://www.renfe.com/content/dam/renfe/es/General/buscadores/javascript/estacionesEstaticas.js?v=5">
  </script>
  <div class="rf-general__content rf-general__content--menu-small">
   <div class="rfheader">
    <rf-header class="rf-header-wrapper hydrated" country="ES" data-search-default='{"dataOrigin":"","dataDestination":""}' domain=".renfe.com" info-login="/vol/header/loginInfo.json" language="es" links='{"languages

In [23]:
# for tag in beautifulSoupText.findAll(True):
#     print(tag.name, " : ", len(beautifulSoupText.find(tag.name).text))

In [24]:
# Hora de Salida:
#  - <div aria-label="Hora de salida"

# Duracion:
#  - <div aria-label="Duración"

# Hora de llegada:
# - <div aria-label="Hora de llegada"

# Tipo de tren:
# - <div aria-label="Tipo de tren" 

# Precio
# - <div class="precio booking-list-element-big-font">

# Differentes precios dependen de:
# posicionproducto="0"/"1"/"2"/


### Extract Departure Times

In [25]:
departure_time_list_raw = beautifulSoupText.find_all("div", class_="booking-list-element-big-font salida displace-text-xs")
departure_time_list = []
for i in range(len(departure_time_list_raw)):
    departure_time = departure_time_list_raw[i].get_text()
    departure_time_list.append(departure_time)

In [26]:
len(departure_time_list)

31

In [27]:
departure_time_list

['\n\t\t\t\t05.50\n\t\t\t',
 ' \n\t\t\t\t06.35\n\t\t\t',
 ' \n\t\t\t\t07.40\n\t\t\t',
 '\n\t\t\t\t08.00\n\t\t\t',
 '\n\t\t\t\t08.25\n\t\t\t',
 ' \n\t\t\t\t08.30\n\t\t\t',
 '\n\t\t\t\t09.00\n\t\t\t',
 '\n\t\t\t\t09.03\n\t\t\t',
 '\n\t\t\t\t09.30\n\t\t\t',
 '\n\t\t\t\t10.00\n\t\t\t',
 ' \n\t\t\t\t10.15\n\t\t\t',
 '\n\t\t\t\t11.00\n\t\t\t',
 ' \n\t\t\t\t12.00\n\t\t\t',
 '\n\t\t\t\t12.00\n\t\t\t',
 '\n\t\t\t\t12.50\n\t\t\t',
 ' \n\t\t\t\t13.00\n\t\t\t',
 '\n\t\t\t\t13.25\n\t\t\t',
 ' \n\t\t\t\t14.00\n\t\t\t',
 '\n\t\t\t\t15.00\n\t\t\t',
 '\n\t\t\t\t15.25\n\t\t\t',
 ' \n\t\t\t\t16.00\n\t\t\t',
 '\n\t\t\t\t16.25\n\t\t\t',
 '\n\t\t\t\t17.00\n\t\t\t',
 '\n\t\t\t\t17.25\n\t\t\t',
 '\n\t\t\t\t18.00\n\t\t\t',
 '\n\t\t\t\t18.25\n\t\t\t',
 '\n\t\t\t\t18.40\n\t\t\t',
 '\n\t\t\t\t19.25\n\t\t\t',
 '\n\t\t\t\t20.00\n\t\t\t',
 '\n\t\t\t\t21.00\n\t\t\t',
 '\n\t\t\t\t21.25\n\t\t\t']

### Extract Arrival Times

In [28]:
arrival_time_list_raw = beautifulSoupText.find_all("div", class_="booking-list-element-big-font llegada")
arrival_time_list = []
for i in range(len(arrival_time_list_raw)):
    arrival_time = arrival_time_list_raw[i].get_text()
    arrival_time_list.append(arrival_time)

In [29]:
arrival_time_list

['\n\t\t\t\t09.10\n\t\t\t',
 '\n\t\t\t\t09.20\n\t\t\t',
 '\n\t\t\t\t10.10\n\t\t\t',
 '\n\t\t\t\t11.12\n\t\t\t',
 '\n\t\t\t\t10.55\n\t\t\t',
 '\n\t\t\t\t15.35\n\t\t\t',
 '\n\t\t\t\t11.45\n\t\t\t',
 '\n\t\t\t\t18.09\n\t\t\t',
 '\n\t\t\t\t20.41\n\t\t\t',
 '\n\t\t\t\t13.17\n\t\t\t',
 '\n\t\t\t\t15.35\n\t\t\t',
 '\n\t\t\t\t13.45\n\t\t\t',
 '\n\t\t\t\t18.35\n\t\t\t',
 '\n\t\t\t\t15.12\n\t\t\t',
 '\n\t\t\t\t15.45\n\t\t\t',
 '\n\t\t\t\t19.40\n\t\t\t',
 '\n\t\t\t\t15.54\n\t\t\t',
 '\n\t\t\t\t17.12\n\t\t\t',
 '\n\t\t\t\t17.45\n\t\t\t',
 '\n\t\t\t\t17.55\n\t\t\t',
 '\n\t\t\t\t19.12\n\t\t\t',
 '\n\t\t\t\t18.55\n\t\t\t',
 '\n\t\t\t\t19.45\n\t\t\t',
 '\n\t\t\t\t19.55\n\t\t\t',
 '\n\t\t\t\t21.12\n\t\t\t',
 '\n\t\t\t\t20.55\n\t\t\t',
 '\n\t\t\t\t21.45\n\t\t\t',
 '\n\t\t\t\t21.55\n\t\t\t',
 '\n\t\t\t\t23.12\n\t\t\t',
 '\n\t\t\t\t23.45\n\t\t\t',
 '\n\t\t\t\t23.55\n\t\t\t']

### Extract Prices

In [30]:
price_list_raw = beautifulSoupText.find_all("div", class_="precio booking-list-element-big-font")
price_list = []
for i in range(len(price_list_raw)):
    price = price_list_raw[i].get_text()
    price_list.append(price)

In [31]:
print(price_list)

['136,10 €', '155,10 €', '136,10 €', '155,10 €', '142,80 €', '161,80 €', '79,60 €', '90,95 €', '185,35 €', '43,25 €', '67,05 €', '74,50 €', '69,00 €', '121,00 €', '140,00 €', '68,20 €', '79,60 €', '136,45 €', '147,80 €', '93,95 €', '107,35 €', '136,45 €', '147,80 €', '215,60 €', '134,20 €', '147,60 €', '179,65 €', '196,35 €', '215,35 €', '125,05 €', '136,45 €', '55,00 €', '102,35 €', '113,70 €', '67,10 €', '80,50 €', '161,80 €', '45,50 €', '56,85 €', '140,00 €', '142,80 €', '161,80 €', '39,80 €', '51,15 €', '94,60 €', '39,00 €', '121,00 €', '140,00 €']


In [ ]:
# Full Table -> //*[@id="listaTrenesTBodyIda"]

# Row 1 -> //*[@id="tren03062"]

# Row 2 -> //*[@id="tren03260"]
# Price Row 2 (Basic) -> //*[@id="trenButton032600"]/div[2]
# Price Row 2 (Choose) -> //*[@id="trenButton032601"]/div[2]
# Price Row 2 (Premium) -> //*[@id="trenButton032602"]/div[2]
